In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "MBZUAI-Paris/Atlas-Chat-9B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)


In [ ]:
import pandas as pd
#importe les commentaires à paraphraser
ds = pd.read_excel(r'/content/sample_data/pos_comments.xlsx')
comments = ds['Comment'].tolist()

In [ ]:
prompt = """
Paraphrase the following sentences in moroccan darija without changing their sentiment, the paraphrased sentence must be different from the original one:

1. "من جهت الفطور المركشي مكاملش اغالي البيصرة والسفنج عادي خاصوا بزززززاف اعلى لفطور المركشي" -> "من ناحية الفطور المراكشي ماشي هوا هدا و غالي   البيصرة والسفنج عاديين هاد الفطور خاصوا بزاف  باش يكون مراكشي"
2. "تبارك الله عيك " -> "برافو عليك",
3. "لفطور  كان غالي ميستهلش داك التمن" -> "داك الثمن غالي بزاف على هاد  الفطور",
4. "عن نفسي استحقر هذه المطاعم وافضل الشعبية" -> "المطاعم الشعبية حسن ماكنحملش هاد النوع د المطاعم",
5. "والى مشيتي لتمارة كين مطعم الاخوين عندو الطواجن والدجاج زوين" -> " فتمارة كين مطعم  الاخوين زوين عندو الطواجن والدجاج واعرين",
6. "احسن طاكوست كليتو في بن جرير عند مرمرة فواكه" -> "مرمرة فواكه لي كاين فبنجرير عندو احسن طاكوس كليت في حياتي",
7. "من نيتك راه غالي بزاف تمن" -> "ماشي معقول هادشي الاثمنة غالية بزاف",
8. "ثمن غالي ديال هد الوجبات الفطور" -> "ثمن الاكل غالي",
9. "هو فعلا كبير بزااااف ولكن ناقصه التنظيم  و التوجيه  للزبائن  ولكن يستحق تمشي ليه الا بغيتي تعرض على ضيافك." -> "بصاح المطعم كبير بزاف ولكن كيعاني من نقص في التنظيم والتوجيه ومع ذلك كيستاهل الزيارة إلا بغيت تكرّم ضيافك.",
10. "مطعم فريكس الي في طريق الشريفية مكان عاءلي بامتياز ويستحق الزيارة ." -> "مطعم فريكس اللي في طريق الشريفية مكان مثالي للعائلات وكيستاهل الزيارة.",
Now, paraphrase this sentence:
"""

def paraphrase_arabic(comment):
    messages = [
    {"role": "user", "content": prompt+comment},
        ]

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True,add_generation_prompt=True)

    outputs = model.generate(**input_ids, max_new_tokens=256)

    return tokenizer.decode(outputs[0])

In [ ]:
augmentedComments =[]
for comment in comments:
    paraphrased_comment = paraphrase_arabic(comment)
    augmentedComments.append(paraphrased_comment)

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2506: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
cleaned_augmentedComments = []
for comment in augmentedComments:
    # Découper le texte après le mot "model"
    comment= comment.replace("<end_of_turn>", "")
    split_text = comment.split("model", 1)  # 1 pour limiter à une seule séparation

    # Vérifier si "model" a été trouvé
    if len(split_text) > 1:
        # Récupérer le texte après "model" et supprimer les espaces inutiles
        cleaned_text = split_text[1].strip()
    else:
        cleaned_text = ""

    cleaned_augmentedComments.append(cleaned_text)

In [ ]:
data = {"Original Comment": comments, "Augmented Comment": cleaned_augmentedComments}
df = pd.DataFrame(data)

df

In [ ]:
df.to_excel('aug_pos_cmts_byAtlasChat.xlsx')